In [1]:
import pandas as pd
import numpy as np
import math as math

In [3]:
raw_data=pd.read_csv('labeled_data_with_ind.csv')
raw_data=raw_data.astype(float)
location_index=['x','y']#the name of column respect location
sigmoid=0.1
arr_lacation_index=[1,2]#location for x\y in data saved in array
f=2
maxcost=1000

#threshold=100


#raw_data=raw_data[0:2]
#raw_data['x'][0],raw_data['y'][0]=0,0
#raw_data['x'][1],raw_data['y'][1]=2*sigmoid,sigmoid*2


In [4]:
max_data=np.array(raw_data[location_index].max(axis=0))
min_data=np.array(raw_data[location_index].min(axis=0))
size_of_cell=list((np.ceil((max_data-min_data)/(2*sigmoid))+1).astype(int))



In [5]:
def mul(a):
    result=1
    for i in a:
        result=result*i
    return int(result)

def dx_to_d1(local,total=size_of_cell):
    d1=0
    for i in range(len(local)): 
        d1+=mul(total[i+1:])*(local[i])
    return int(d1)

def d1_to_dx(d1,total=size_of_cell):
    dx=[]
    for i in range(len(total)):
        local=mul(total[i+1:])
        a=int(np.floor(d1/local))
        dx.append(a)
        d1=d1%local
    return list(dx)
    

In [6]:
#partition data into cell
def data_to_cell(data,sigmoid,location_index,size_of_cell=size_of_cell):
    
    max_data=np.array(data[location_index].max(axis=0))
    min_data=np.array(data[location_index].min(axis=0))
    #size_of_cell=list(np.ceil((max_data-min_data)/(2*sigmoid)).astype(int))
    
    #initial 
    #number in cell/ 1d index of cell
    cell=[[0,i] for i in range(mul(size_of_cell))]
    
    for i in range(len(data)):
        location=data[location_index][i:i+1]
        cell_index=np.array(np.floor((location-min_data)/(2*sigmoid)).astype(int))[0]
        cell_index=dx_to_d1(cell_index,size_of_cell)
        local_data=list(np.array(data[i:i+1]))
        #print(cell_index)
        cell[cell_index].append(local_data)
        cell[cell_index][0]+=1
    return cell


In [7]:
#estimate cost for one cell
#given by papper 
#f is the average fanout of the R-tree
#NS in point of  space rectangle S, consist of all the cell
#n is the number of dimensio
def cell_cost(data,f,NS,n):
    if data[0]!=0:
        h=int(1+math.ceil(math.log((1.0*NS/f),f)))
        DA=1
        for i in range(h):
            j=i+1
            DA+=(1+(1.0*data[0]/(f**j))**(1.0/n))**n
        w=DA*data[0]
    else:
        w=0
    return w
        
def reg_cost(data,f,location_index):
    n= len(location_index)
    
    NS=0
    for d in data:
        NS+=d[0]
    
    WS=0
    if NS!=0:
        for local_data in data:
            W_c=cell_cost(local_data,f,NS,n)
            WS+=0+W_c
    return WS
        
def reg_cost2(data,f,location_index):
    #use the number of point as cost
    #f is useless, just to make simialr with reg_cost
    NS=0
    for d in data:
        NS+=d[0]
    return NS
    
def reg_cost3(data,location_index):
    #use the number of point as cost
    #f is useless, just to make simialr with reg_cost
    NS=0
    for d in data:
        NS+=d[0]
    return NS    


def point_to_data(data,point,size_of_cell=size_of_cell):
    #from two point to get total data based on cell size
    down_cell=d1_to_dx(point[0],size_of_cell)
    up_cell=d1_to_dx(point[1],size_of_cell)    
    total_data=meshgrid_kd(down_cell,up_cell)
    total_data_index=[dx_to_d1(i,size_of_cell) for i in total_data] 
    data=[data[index] for index in total_data_index]
    return data

In [8]:
def meshgrid_kd(down_cell,up_cell):
    #give the mesh grid for different dimension
    a=[[]]
    for i in range(len(down_cell)):
        b=[[j] for j in range(down_cell[i],up_cell[i]+1) ]
        d=[]
        for j in b:
            d+=[k+j for k in a]
        a=d
    return a

#cost based binary spllit
#for data , we only use 2 index(two cornor of regtangle, taht is,max/min id index)to respect it, insead of all data
#so that we do not have to partition or copy raw data evertime, just up data the cornor
#up data consist of two index, [min, max]
def split_binary(up_data,location_index,f):
    down_cell=d1_to_dx(up_data[0],size_of_cell)
    up_cell=d1_to_dx(up_data[1],size_of_cell)
       
    total_data=meshgrid_kd(down_cell,up_cell)
    total_data_index=[dx_to_d1(i,size_of_cell) for i in total_data] 
    
    mincost_di=float("inf")
    #use one dim to cut data
    for i in range(len(location_index)):        
        #init change date and S1,S2
        change_down=down_cell[:]
        change_up=up_cell[:]
        #print(down_cell,up_cell)
        left_data_index=[]
        right_data_index=total_data_index[:]
        for j in range(down_cell[i], up_cell[i]):  
            #print(down_cell,up_cell)
            change_down[i]=j
            change_up[i]=j
            #print(down_cell,up_cell)
            change_column=meshgrid_kd(change_down,change_up)
            change_column_index=[dx_to_d1(column_index,size_of_cell) for column_index in change_column] 
            for element in change_column_index:
                left_data_index.append(element)
                right_data_index.remove(element)
            #print(len(right_data_index))
            #print(len(left_data_index))
            left_data=[data[index_left] for index_left in left_data_index]
            right_data=[data[index_right] for index_right in right_data_index]
            cost_di=abs(reg_cost(left_data,f,location_index)-reg_cost(right_data,f,location_index))
            if cost_di<mincost_di:
                mincost_di=cost_di
                best_left_data_index,best_right_data_index= left_data_index[:],right_data_index[:]
    
    b_l_d_down_cell=min( best_left_data_index)
    b_l_d_up_cell=max( best_left_data_index)
    b_r_d_down_cell=min( best_right_data_index)
    b_r_d_up_cell=max( best_right_data_index)
    
    left=[b_l_d_down_cell,b_l_d_up_cell]
    right=[b_r_d_down_cell,b_r_d_up_cell]
    return left,right
            
            
            
            
            
            
            

In [9]:
def partition(data,maxcost,location_index,cost_function=reg_cost,f=f):
    #default cost function
    #data has been cut into cell before
    Su=[0,len(data)-1]
    queue=[Su] 
    partition=[]
    
    while len(queue)>0:
        S=queue[0]
        queue=queue[1:]
        #print(S)
        S_data=point_to_data(data,S,size_of_cell=size_of_cell)
        if cost_function(S_data,f,location_index)>maxcost:
            S1,S2=split_binary(S,location_index,f)
            queue.append(S1)
            queue.append(S2)
        else:
            partition.append(S)

    return partition

                
            
            
    
    


In [11]:
def choose_shell(down_cell,up_cell):
    #choose only the outermost layer
    result=[]
    for i in range(len(down_cell)):
        local_down=down_cell[:]
        local_up=up_cell[:]
        local_down[i]=up_cell[i]
        result=result+meshgrid_kd(local_down,local_up)
        local_down=down_cell[:]
        local_up=up_cell[:]
        local_up[i]=down_cell[i]   
        result=result+meshgrid_kd(local_down,local_up)
    #result=list(set(result))
    result = list(set([tuple(t) for t in result]))
    result = [list(v) for v in result]
    return result
    
def restrict_cell(down_cell,size_of_cell=size_of_cell):
    min_cell=list(np.zeros(len(size_of_cell)))
    max_cell=list(np.array(size_of_cell)-np.ones(len(size_of_cell)).astype(int))
    down_cell=max(down_cell,min_cell)
    down_cell=min(max_cell,down_cell)
    down_cell=map(int, down_cell)
    return list(down_cell)


def data_outside(point,arr_lacation_index=arr_lacation_index,size_of_cell=size_of_cell,data=data,k=1,globle_min_point=min_data,sigmoid=sigmoid):
    #use point to get data in next shell(not in the small regtangle), 1-sigmoid扩展矩形部分，不属于中央的矩形
    #the extendece of reg = k*sigmoid ,and ,k=1<2
    down_cell=d1_to_dx(point[0],size_of_cell)
    up_cell=d1_to_dx(point[1],size_of_cell)
    
    data_in_next_cell=[]
    big_down_cell=list(np.array(down_cell)-1)
    big_up_cell=list(np.array(up_cell)+1)
    big_down_cell=restrict_cell(big_down_cell)
    big_up_cell=restrict_cell(big_up_cell)
    
    shell=choose_shell(big_down_cell,big_up_cell)
    shell_index=[dx_to_d1(item) for item in shell]
    
    shell_data=[data[i] for i in shell_index]
    #shell data = 2-sigmoid 
    shell_data_list=cell_to_list(shell_data)
    
    #consider whither in k_sigmoid
    min_point=globle_min_point+(2*np.array(down_cell)-k)*sigmoid
    max_point=globle_min_point+(2*np.array(up_cell)+k+2)*sigmoid
    
    result=[]
    for i in shell_data_list:
        record=i[0]
        item=[record[j] for j in arr_lacation_index]
        a=min_point<item
        b=item<max_point
        if np.prod(a) and np.prod(b):
            result.append(i)
            
        
        
    
    return result

def data_inside(point,data=data):
    #to list data
    cell_data=point_to_data(data,point,size_of_cell=size_of_cell)
    result=cell_to_list(cell_data)
    return result
        

    
def cell_to_list(data):
    #data is the size of cell based kinds to a list  
    result=[]
    for item in data:
        local_data=item[2:]
        result=result+local_data
    return result

 
    
    
    

In [12]:
def bod_to_dataset(data_bod):
    index_of_part=0
    total_data=[]
    for point in data_bod:
        local=[]
        local=data_outside(point)+data_inside(point)
        local=[list(record[0]) for record in local]
        local=list(set([tuple(record) for record in local]))
        local = [[index_of_part]+list(v)[:] for v in local]
        index_of_part=index_of_part+1
        total_data=total_data+local
    return total_data
        

In [31]:
#maxcost=3000
data=data_to_cell(raw_data,sigmoid,location_index)
data_bod=partition(data,maxcost,location_index)
result=bod_to_dataset(data_bod)
new_head=['partition']+list(raw_data.head(0))
final=pd.DataFrame(columns=new_head,data=result)

In [32]:
print(len(partition(data,maxcost,location_index)))
print(len(final))

8
968


In [33]:
final.to_csv('data_partition——8p.csv')